In [3]:
import re
import nltk
import pickle5 as pickle
import numpy as np
import pandas as pd
import requests
from nltk.corpus import stopwords
nltk.download('stopwords')
from itertools import groupby
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Embedding, Dense, Flatten, LSTM, GRU, GlobalAveragePooling1D, Bidirectional, Dropout, BatchNormalization, Input, Conv2D, MaxPool2D, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load label, tokenizer with max_len, and models
label_dict = {'istj': 0, 'isfj': 1, 'infj': 2, 'intj': 3, 'istp': 4, 'isfp': 5, 'infp': 6, 'intp': 7,
              'estp': 8, 'esfp': 9, 'enfp': 10, 'entp': 11, 'estj': 12, 'esfj': 13, 'enfj': 14, 'entj': 15}
with open('./models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
max_len = 4491
word_index = tokenizer.word_index
model = load_model('./models/model_conv_net.h5')
model.summary()
print("Done.")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4491)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 4491, 64)          15050944  
                                                                 
 reshape_1 (Reshape)         (None, 4491, 64, 1)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 4491, 64, 32)      160       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 2245, 32, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 2245, 32, 32)      4128      
                                                           

In [9]:
# 모델 활용시 필요한 함수 정의
# 전처리 함수
def preprocess_date(line):
    pattern = r'-{15}.*-{15}'
    return re.sub(pattern, '', line) 

def preprocess_text(texts):
    stop_words = set(stopwords.words('english'))    
    texts = texts.apply(lambda x: re.sub(r'@\w+\s?', '', x)) # remove usernames
    texts = texts.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x)) # remove links
    texts = texts.apply(lambda x: x.lower()) # convert to lowercase
    texts = texts.apply(lambda x: re.findall(r'\b\w+\b', x)) # split into individual words     
    # texts = texts.apply(lambda x: [word for word in x if word not in stop_words]) # remove stop words
    return texts

# 한->영 함수
def translate_text(text, source_language='ko', target_language='en'):
    url = 'https://translate.googleapis.com/translate_a/single?client=gtx&sl={}&tl={}&dt=t&q={}'
    response = requests.get(url.format(source_language, target_language, text)).json()
    try:
        translation = response[0][0][0]
    except (IndexError, TypeError):
        print(f"Translation failed for text: {text}")
        translation = ""
    return translation

# 텍스트 -> mbti 함수
def predict_mbti(text):
    texts = preprocess_text(pd.Series([text]))
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    prediction = model.predict(padded_sequences, verbose=0)[0]
    labels = label_dict.keys()
    result = {label: prediction[idx] for idx, label in enumerate(labels)}
    return result

# 카카오톡 대화파일 -> 사용자별 mbti 함수
def predict_mbti_kakaotalk(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Remove date and time information
    pattern = r'^\d{4}-\d{2}-\d{2}, \d{1,2}:\d{2} [APap][Mm] - '
    lines = [re.sub(pattern, '', line) for line in lines]
    lines = [preprocess_date(line) for line in lines]
    
    # Get chat room name and saved date
    chat_room_name = lines[0].strip()
    saved_date = lines[1].strip()
    print(f"Chat Room: {chat_room_name}, Saved Date: {saved_date}")

       # Group messages by user
    user_messages = {}
    current_user = None
    pattern = r'^\[(.*?)\] '  # [사용자명] 패턴
    time_pattern = r'\[(오전|오후)\s\d+:\d+\]\s'
    for line in lines[2:]:
        user_match = re.search(pattern, line)
        if user_match:
            current_user = user_match.group(1)
            if current_user not in user_messages:
                user_messages[current_user] = []
            # Add message after user name
            message = re.sub(pattern, '', line).strip()
            message = re.sub(time_pattern, '', message)  # Remove time information
            user_messages[current_user].append(message)
        elif current_user is not None and line.strip() != '':
            # Append message to previous user
            user_messages[current_user][-1] += ' ' + re.sub(time_pattern, '', line.strip())  # Remove time information

    # Translate and predict MBTI for each user
    mbti_results = {}
    for user, messages in user_messages.items():
        print(f"\nProcessing messages for user: {user}")
        print(f"Messages: {messages}")
        translated_messages = [translate_text(message) for message in messages]
        prediction = predict_mbti(' '.join(translated_messages))
        top_k = sorted(prediction.items(), key=lambda x: x[1], reverse=True)[:]
        mbti_results[user] = top_k

    # Print MBTI results by user
    for user, results in mbti_results.items():
        print(f"{user}: {results}") 

In [10]:
predict_mbti_kakaotalk('./KakaoTalk/KakaoTalk_20230509_1655_31_857_현명.txt')

Chat Room: 현명 님과 카카오톡 대화, Saved Date: 저장한 날짜 : 2023-05-09 16:55:33

Processing messages for user: 현명
Messages: []

Processing messages for user: 김현우
Messages: []
현명: [('infp', 0.12568401), ('infj', 0.10242531), ('intp', 0.07970657), ('intj', 0.075053334), ('enfj', 0.07170661), ('enfp', 0.07016323), ('isfj', 0.07000509), ('istp', 0.06441566), ('isfp', 0.05745092), ('istj', 0.056164563), ('entp', 0.053523406), ('entj', 0.050036132), ('esfp', 0.046412587), ('esfj', 0.027513064), ('estp', 0.027047968), ('estj', 0.02269149)]
김현우: [('infp', 0.12568401), ('infj', 0.10242531), ('intp', 0.07970657), ('intj', 0.075053334), ('enfj', 0.07170661), ('enfp', 0.07016323), ('isfj', 0.07000509), ('istp', 0.06441566), ('isfp', 0.05745092), ('istj', 0.056164563), ('entp', 0.053523406), ('entj', 0.050036132), ('esfp', 0.046412587), ('esfj', 0.027513064), ('estp', 0.027047968), ('estj', 0.02269149)]


In [7]:
# (1) 성능 개선의 방법
# 전처리 고도화
# 모델 고도화 (컨볼루션 트랜스포머 완료)
# Transformer, BERT (GPT알고리즘) 모형 사용하기 (완료)
# 기존 학습되어 있는 언어모델 레이어 불러와서 사용하기

# (2) 카카오톡 대화 내용 넣기 (완료)
# 번역 (완료)
# 복수의 문장이 들어가서 MBTI 출력되게 구현하기 (완료)
# 정규화 개선해야함.)

In [8]:
predict_mbti_kakaotalk('./KakaoTalk/KakaoTalk_20230504_1223_50_096_group.txt')

Chat Room: 로봇 과제 공지 전달방 님과 카카오톡 대화, Saved Date: 저장한 날짜 : 2023-05-04 12:23:58

Processing messages for user: 남다현
Messages: ['2. 보드, 노트북, 센서만 제공된다면 추가로 구매할 하드웨어는 150-300 안에서 다 해결될 것 같습니다', '3. 넵!!', '장민준 / 2019145094 / 010-2876-0427 / pawpaw0427@gmail.com', '최서연 / 2019145030 /  010-2127-8361 / seoyeon99@yonsei.ac.kr', '기인호 / 해당없음', '좋은 하루 보내세요!', '오늘 미팅의 경우 대면으로 진행될까요?', '10분 정도 늦게 시작하는걸로 해서 대면으로 진행하는 것은 어떨까요? 번복드려 죄송합니다 😅', '하드웨어 받은 후 정리되는대로 보내드리겠습니다 : )', '--------------- 2023년 3월 21일 화요일 ---------------', '학부 수업 때문에 15시 이후쯤 가능할 것 같습니다! 혹시 이때도 가능하시다면 찾아봬도 될까요?', '저희가 아직 주제를 모두 정하지 못한 상태인지라, 컨셉 관련 키페이퍼&데이터셋 등등 내일 당장 찾아보는 것이 조금 어려울 듯 싶습니다!', '혹시 다음주까지 시간을 가지고 컨셉 확정을 해봐도 될까요?', '주중 업무시간대로 다시 한번 일정 잡아서 바로 톡 드리겠습니다!', '필요 내용은 아래와 같습니다.', '카메라의 경우 로지텍 C920 모델 채택 예정이고 마운트 방식은 프린팅 혹은 알루미늄 프로파일로, 아직 미정인 상태입니다.', '팬/틸트의 경우, 아래의 두가지 방법을 논의중에 있습니다. (위고와 미팅 후 결정해야할 것 같습니다!)', '1. 다이나믹셀 + u2d2 모듈 사용 (다이나믹셀, u2d2 키트, 브라켓 구매 필요)', '2. 원격 제어 가능한 팬/틸트 모듈 구매', '지난주에 말씀드렸던 구매 필요 항목들 리스트 송부드립니다~!', '저는 아마 


Processing messages for user: 최서연
Messages: ['--------------- 2023년 4월 1일 토요일 ---------------', '1. 배트캠 ip를 할당해줄 공유기가 필요합니다.', '2. 10AWG 사이즈의 커넥터가 추가로 필요할 것 같습니다.', '--------------- 2023년 4월 23일 일요일 ---------------']

Processing messages for user: 장민준
Messages: ['--------------- 2023년 4월 11일 화요일 ---------------', '해당 websocket server를 서버컴퓨터에서 실행하고 로봇의 nuc에서 서버의 ip를 항햐 str 형태의 데이터를 전달하도록 구성되어 있으며 전달하는 데이터는 entity 추가를 통해 확장 가능하니 필요한 데이터가 생길 때마다 추가하는 방식으로 전개할 수 있을 것이라 생각됩니다.', '사용 방법에 대한 설명은 README에 작성해두었습니다']

Processing messages for user: 인호
Messages: ['계정 아이디는 smi로 되어있습니다', '--------------- 2023년 3월 29일 수요일 ---------------', '배트캠 팬틸트를 위해서 브라켓을 만들어야합니다', '배트캠 도면이나 치수라도 적힌 자료 아무거나 받아볼 수 있을까요?', '--------------- 2023년 4월 12일 수요일 ---------------', 'scoutmini 제어 코드도 구현하여 마찬가지로 클래스로 만들어 놨습니다', '현재 다현이가 팬틸트 모듈에 대한 클래스 만들고 있으며', 'QR은 팬틸트 브라켓 도착하는대로 팬틸트 코드와 함께 Batcam쪽 클래스로 합칠 예정입니다!']

Processing messages for user: 질문 사항
Messages: ['- 초기 프로토타입 설계 및 세팅한 업체 (사용 하드웨어 구성도 및 전원분배 관련 정보 필요)', '- 사용 